In [ ]:
# Import des librairies 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
import csv

In [ ]:
# Connexion a Glassdoor

# Ouverture de chrome et accès a Glassdoor

driver = webdriver.Chrome()
sleep(2)
url = 'https://www.glassdoor.fr/index.htm'
driver.get(url)
print('Initialisation driver')
sleep(2)

sign_in = driver.find_element(By.CLASS_NAME, "LockedHomeHeaderStyles__signInButton")
sign_in.click()

# Import username et password. Il faut un fichier local avec les infos de connexions
credential = open('credentials_glassdoor.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('Succès import credentials')
sleep(2)

#Insertion des credentials
#Il arrive que cette partie ne fonctionne pas, de manière aléatoire. Il suffit alors de cliquer manuellement sur l'encars login et de relancer le script.
#Il doit s'agir d'une mesure anti scraping

email_field = driver.find_element(By.NAME, "username")
email_field.send_keys(username)
print('Succès insertion email')
sleep(3)

submit = driver.find_element(By.NAME, "submit")
submit.click()
sleep(2)

password_field = driver.find_element(By.NAME, "password")
password_field.send_keys(password)
print('Succès insertion password')
sleep(2)

print('Connexion a Glassdoor avec succès')

In [ ]:
# Scan des 1000 premières boites qui proposent des postes de Data engineer. Pour avoir un echantillon si grand, on accepte des variantes tels que Data analyste
# Pour une raison que j'ignore, le script s'arrête de manière aléatoire car glassdoor refuse parfois de charger la page suivante d'entreprise. Il doit s'agir d'une mesure anti scraping 

url = 'https://www.glassdoor.fr/Salaires/france-date-engineer-salaire-SRCH_IL.0,6_IN86_KO7,20.htm?clickSource=searchBtn'
driver.get(url)
input_page = int(input('Combien de pages voulez vous scraper?: '))
data_divs = []

with open('output_glassdoor.csv', 'a',  newline = '') as file_output:
    headers = ['Company Name', 'Job Title', 'Salary', 'Min salary', 'Max salary', 'Fourchette']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()

    for page in range(input_page):
        page_source = BeautifulSoup(driver.page_source)
        data_current_page = page_source.find_all('div', class_ = 'py css-17435dd')
        for div in data_current_page:
            company = div.find('a',{'class':'css-f3vw95 e1aj7ssy3'}).getText().strip()
            job = div.find('span',{'class':'m-0 css-18stkbk'}).getText().strip()
            salary = div.find('div',{'class':'d-flex align-items-baseline'}).find('h3').getText().encode("ascii",'ignore').decode("utf-8")
            min_salary = div.find_all('p',{'class':'m-0 css-1vkj9it'})[0].getText().encode("ascii",'ignore').decode("utf-8")
            max_salary = div.find_all('p',{'class':'m-0 css-1vkj9it'})[1].getText().encode("ascii",'ignore').decode("utf-8")
            fourchette = div.find_all('p',{'class':'m-0 css-1vkj9it'})[2].getText().encode("ascii",'ignore').decode("utf-8")
            data_divs.append(company + ',' + job + ',' + salary + ',' + min_salary + ',' + max_salary + ',' + fourchette)
            writer.writerow({headers[0]:company, headers[1]:job, headers[2]:salary, headers[3]:min_salary, headers[4]:max_salary, headers[5]:fourchette})
        
        sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #Scroll a la fin de la page
        sleep(3)
        next_button = driver.find_element(By.CLASS_NAME, "nextButton")
        driver.execute_script("arguments[0].click();", next_button)
        sleep(2)

print('Scraping des divs avec succès')